In [116]:
import pandas as pd
import json
import glob
import os
import requests
from datetime import datetime, timedelta
import time

In [22]:
def readGlobDf(filename):
    df = pd.concat([pd.read_json(file, lines = True) for file in 
               glob.glob(filename.replace(".json","*.json"))])
    return df

In [124]:
df_python_repos = pd.DataFrame(columns = ['repo_name','repo_id','created_at','stars'])

In [156]:
token = os.environ['token']
username = 'liaochris'


In [161]:
api_url_calls = []
URL = 'https://api.github.com/search/repositories?q=language:Python+created:{}..{}&per_page=100&page={}'
since = datetime(2008, 1, 1, 0, 0)  # Since 30 days ago
until = since + timedelta(days=30)   # Until 29 days ago 

In [162]:
while until < datetime.today():
    # Update dates for the next search
    for i in range(10):
        day_url = URL.format(since.strftime('%Y-%m-%d'), until.strftime('%Y-%m-%d'), i+1)
        api_url_calls.append((day_url, since, until))
    since = until
    until = since + timedelta(days=30)

In [158]:
for day_url in api_url_calls:
    try:
        r = requests.get(day_url[0], auth=(username, token))
        r_json = r.json()
        repos = r_json['items']
        date_data = [[ele['full_name'], ele['id'], ele['created_at'], ele['stargazers_count']] for ele in repos]
        df_repos_date = pd.DataFrame(date_data,columns = ['repo_name','repo_id','created_at','stars'])
        df_python_repos = pd.concat([df_repos_date, df_python_repos]).drop_duplicates()
        print(f'Repositories created between {day_url[1]} and {day_url[2]}: {r_json.get("total_count")}')
        time.sleep(1)
    except:
        time.sleep(30)
        print("Pausing")
        r = requests.get(day_url[0], auth=(username, token))
        r_json = r.json()
        repos = r_json['items']
        date_data = [[ele['full_name'], ele['id'], ele['created_at'], ele['stargazers_count']] for ele in repos]
        df_repos_date = pd.DataFrame(date_data,columns = ['repo_name','repo_id','created_at','stars'])
        df_python_repos = pd.concat([df_repos_date, df_python_repos]).drop_duplicates()
        print(f'Repositories created between {day_url[1]} and {day_url[2]}: {r_json.get("total_count")}')
        time.sleep(1)

Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-01 00:00:00 and 2008-01-31 00:00:00: 0
Repositories created between 2008-01-31 00:00:00 and 2008-03-01 00:00:00: 14
Repositories created between 2008-01-31 00:00:00 and 2008-03-01 00:00:00: 14
Repositories created between 2008-01-31 00:00:00 and 2008-03-01 00:00:00: 14
Repositor

In [173]:
df_python_repos.reset_index(drop = True).sort_values('stars', ascending = False)

,repo_name,repo_id,created_at,stars
93967,public-apis/public-apis,54346799,2016-03-20T23:49:42Z,286499
83288,donnemartin/system-design-primer,83222441,2017-02-26T16:15:28Z,249091
115290,vinta/awesome-python,21289110,2014-06-27T21:00:06Z,201338
91053,TheAlgorithms/Python,63476337,2016-07-16T09:44:01Z,176690
71648,jackfrued/Python-100-Days,123458551,2018-03-01T16:05:52Z,148533
...,...,...,...,...
165816,voxlet/mixi-atom-python,482129,2010-01-21T13:36:26Z,0
165815,Tillsten/PyTournament,488727,2010-01-26T03:45:56Z,0
165814,jgallen23/EZTVbot,481570,2010-01-21T01:57:16Z,0
165813,rdno/Asma,486790,2010-01-24T21:17:10Z,0


In [177]:
df_python_repos['repo_id'].to_csv('data/inputs/python_languages.csv')